In [155]:
import numpy as np
from scipy.spatial import distance_matrix
from gurobipy import *
#from scipy.spatial import ConvexHull
#from shapely.geometry import Polygon, Point
from numpy import random
import time
import pandas as pd
from itertools import zip_longest   # zip 함수 길이 다를 경우
from haversine import haversine

In [156]:
#행정동 = '장위2동' # 3개 -> 1
#행정동 = '돈암1동' # 5개 -> 2
#행정동 = '망우3동' # 4개 -> 1
행정동 = '중곡3동' # 2개 -> 1

In [157]:
입지후보지 = pd.read_csv('수정_최종후보입지.csv', encoding='EUC-KR')
입지후보지 = 입지후보지[입지후보지['행정동'] == 행정동]

버스 = pd.read_csv('입지4_버스.csv', encoding='utf-8')
#버스 = 버스[버스['행정동'].isin(행정동리스트)]
버스_points = np.array([list(i) for i in zip(버스['X좌표'], 버스['Y좌표'])])

지하철 = pd.read_csv('입지4_지하철.csv', encoding='utf-8')
#지하철 = 지하철[지하철['행정동'].isin(행정동리스트)]
지하철_points = np.array([list(i) for i in zip(지하철['경도'], 지하철['위도'])])

주차장 = pd.read_csv('입지4_주차장.csv', encoding='utf-8')
#주차장 = 주차장[주차장['행정동'].isin(행정동리스트)]
주차장_points = np.array([list(i) for i in zip(주차장['경도'], 주차장['위도'])])

주택 = pd.read_csv('입지4_주택.csv', encoding='utf-8')
#주택 = 주택[주택['행정동'].isin(행정동리스트)]
주택_points = np.array([list(i) for i in zip(주택['위도'], 주택['경도'])])   # 위도 경도 바뀜

In [158]:
X = list(버스['X좌표']) + list(지하철['경도']) + list(주차장['경도']) + list(주택['위도'])     # 주택 위도 경도 이름 바뀜
Y = list(버스['Y좌표']) + list(지하철['위도']) + list(주차장['위도']) + list(주택['경도'])

In [159]:
points = np.array([list(i) for i in zip(X, Y)])

In [160]:
전체w = points.shape[0]
버스w = 버스.shape[0]
지하철w = 지하철.shape[0]
주차장w = 주차장.shape[0]
주택w = 주택.shape[0]

In [161]:
# 가중치 by AHP 분석(데이터 불균형 * 노인 통행수단 선호도) 
m1 = (전체w-버스w)/전체w * 0.278 
m2 = (전체w-지하철w)/전체w * 0.136 
m3 = (전체w-주차장w)/전체w * 0.193 
m4 = (전체w-주택w)/전체w * 0.392 

In [162]:
# haversine -> meter 단위로 수정
def mclp3(버스_points, 지하철_points, 주차장_points, 주택_points, points, K, radius):
    
    """
    Solve maximum covering location problem
    Input:
        points: 버스정류장, 지하철역 위치 좌표 등 (기타 인근에 있으면 좋은 시설 좌표)
        K: 배치할 노인놀이터의 수
        radius: 반경 (노인들이 이동하기 적합한 거리)
        M: generate_candidate_sites 함수에서 생성할 random 좌표 수 (임의의 노인놀이터 수)
        the ConvexHull wrapped by the polygon
    Return:
        opt_sites: locations K optimal sites, Numpy array in shape of [K,2]
        f: the optimal value of the objective function
    """
    
    print('  Number of points %g' % points.shape[0])
    print('  K %g' % K)
    print('  Radius %g' % radius)

    start = time.time()
    sites = np.array([list(i) for i in zip(입지후보지['x좌표'], 입지후보지['y좌표'])])
    J = sites.shape[0]                                         # 후보지 수
    
    # 수요지점 수
    A = 버스_points.shape[0]
    B = 지하철_points.shape[0]
    C = 주차장_points.shape[0]
    D = 주택_points.shape[0]
    
    # 후보지와 수요지점 간 거리 계산
    D1 = []
    for i in 버스_points:
        site = []
        for j in sites:
            site.append(haversine(i, j)*1000)
        D1.append(site)
    D1 = np.array(D1)
    
    D2 = []
    for i in 지하철_points:
        site = []
        for j in sites:
            site.append(haversine(i, j)*1000)
        D2.append(site)
    D2 = np.array(D2)    
    
    D3 = []
    for i in 주차장_points:
        site = []
        for j in sites:
            site.append(haversine(i, j)*1000)
        D3.append(site)
    D3 = np.array(D3)
    
    D4 = []
    for i in 주택_points:
        site = []
        for j in sites:
            site.append(haversine(i, j)*1000)
        D4.append(site)
    D4 = np.array(D4)
    
    for i in [D1, D2, D3, D4]:
        mask1 = i<=radius
        i[mask1]=1                                                 # 반경 내 속하면 1, 아니면 0
        i[~mask1]=0

    m = Model()
    x1, x2, x3, x4 = {}, {}, {}, {}
    y = {}
    
    # 수요지점 변수 추가
    for i in range(A):                                       
        x1[i] = m.addVar(vtype=GRB.BINARY, name="x1%d" % i)
    for i in range(B):                                       
        x2[i] = m.addVar(vtype=GRB.BINARY, name="x2%d" % i)
    for i in range(C):                                       
        x3[i] = m.addVar(vtype=GRB.BINARY, name="x3%d" % i)
    for i in range(D):                                       
        x4[i] = m.addVar(vtype=GRB.BINARY, name="x4%d" % i)
    
    for j in range(J):
        y[j] = m.addVar(vtype=GRB.BINARY, name="y%d" % j)     # 후보지 변수 추가

    m.update()
    m.addConstr(quicksum(y[j] for j in range(J)) == K)        # 후보지 제약 조건

    # 수요지점 제약 조건
    for i in range(A): 
        m.addConstr(quicksum(y[j] for j in np.where(D1[i]==1)[0]) >= x1[i])
    for i in range(B): 
        m.addConstr(quicksum(y[j] for j in np.where(D2[i]==1)[0]) >= x2[i])
    for i in range(C): 
        m.addConstr(quicksum(y[j] for j in np.where(D3[i]==1)[0]) >= x3[i])
    for i in range(D): 
        m.addConstr(quicksum(y[j] for j in np.where(D4[i]==1)[0]) >= x4[i])
    
    # 목적함수 수정
    res=[]
    for a,b,c,d in zip_longest(range(A),range(B),range(C),range(D), fillvalue=0):
        w1=m1;w2=m2;w3=m3;w4=m4
        if a==b==c==d==0:
            w1=m1;w2=m2;w3=m3;w4=m4
        else:
            if b==0:
                w2=0
            if c==0:
                w3=0
            if d==0:
                w4=0
        res.append(w1*x1[a] + w2*x2[b] + w3*x3[c] + w4*x4[d])

    m.setObjective(quicksum(i for i in res),GRB.MAXIMIZE)
    
    m.setParam('OutputFlag', 0)
    m.optimize()
    end = time.time()
    print('----- Output -----')
    print('  Running time : %s seconds' % float(end-start))
    print('  Optimal coverage points: %g' % m.objVal)
    
    solution = []
    if m.status == GRB.Status.OPTIMAL:
        for v in m.getVars():
            # print v.varName,v.x
            if v.x==1 and v.varName[0]=="y":
                solution.append(int(v.varName[1:]))
    opt_sites = sites[solution]
    return opt_sites,m.objVal

In [163]:
opts_sites, mobjVal = mclp3(버스_points, 지하철_points, 주차장_points, 주택_points, points, 1, 300)
opts_sites

  Number of points 205
  K 1
  Radius 300
----- Output -----
  Running time : 0.018946170806884766 seconds
  Optimal coverage points: 0.821863


array([[127.0848034 ,  37.56777776]])

In [164]:
후보지 = pd.DataFrame(opts_sites, columns=['경도','위도'])
후보지.to_csv('중곡3동_후보지.csv', index=False, encoding='cp949')

수요지점 = pd.DataFrame(points, columns=['경도','위도'])
수요지점.to_csv('중곡3동_수요지점.csv', index=False, encoding='cp949')

버스_수요지점 = pd.DataFrame(버스_points, columns=['경도','위도'])
버스_수요지점.to_csv('중곡3동_버스수요지점.csv', index=False, encoding='cp949')

지하철_수요지점 = pd.DataFrame(지하철_points, columns=['경도','위도'])
지하철_수요지점.to_csv('중곡3동_지하철수요지점.csv', index=False, encoding='cp949')

주차장_수요지점 = pd.DataFrame(주차장_points, columns=['경도','위도'])
주차장_수요지점.to_csv('중곡3동_주차장수요지점.csv', index=False, encoding='cp949')

주택_수요지점 = pd.DataFrame(주택_points, columns=['경도','위도'])
주택_수요지점.to_csv('중곡3동_주택수요지점.csv', index=False, encoding='cp949')